<font color="#CC3D3D"><p>
# [Competition] Building a `Linear Model` with `Pipeline+Optuna`

<font color="blue"><p>
#### LM 모형 구축절차
1. 수치형 피처
 - 결측값처리: SimpleImputer(strategy=`???`)
 - 이상값처리: FunctionTransformer((remove_outlier, kw_args={'q':`???`})))
 - 스케일링: PowerTransformer()
2. 범주형 피처
 - 결측값처리: SimpleImputer(strategy="most_frequent")
 - 인코딩: OneHotEncoder(handle_unknown="ignore", sparse=True)
3. 공통
 - Feature Selection: SelectPercentile(percentile=`???`)
 - Modeling: Ridge(alpha=`???`)
 - Hyperparametor Optimization: `OptunaSearchCV`
 - OOF Prediction   

In [1]:
LM_VERSION = 1.0

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostRegressor
from datetime import timedelta
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.integration.lightgbm import LightGBMTunerCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder, WOEEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [4]:
X_train['근무형태'] = X_train['근무형태'].fillna('신입')
X_test['근무형태'] = X_test['근무형태'].fillna('신입')

In [5]:
X_train['어학시험'] = X_train['어학시험'].fillna('없음')
X_train = X_train.replace({'어학시험' :' '},'없음')
X_test['어학시험'] = X_test['어학시험'].fillna('없음')
X_test = X_test.replace({'어학시험' :' '},'없음')

In [6]:
df=pd.concat([X_train, X_test])

df['대학전공']=df['대학전공'].replace(
{
    '경영':'경영학',
    '경영학':'경영학',
    '경영학 전공':'경영학',
    '경영학과':'경영학',
    '경영학과 (주간)':'경영학',
    '경영학과(주간)':'경영학',
    '경영학부':'경영학',
    '경영학부 경영학':'경영학',
    '경영학부 경영학 전공':'경영학',
    '경영학사':'경영학',
    '경영정보':'경영정보학',
    '경영정보학과':'경영정보학',
    'MIS(경영정보시스템)':'경영정보학',
    '인터넷경영정보':'경영정보학',
    '경영정보(MIS)':'경영정보학',
    '경영정보과':'경영정보학',
    '경영정보시스템':'경영정보학',
    '경영정보과학부':'경영정보학',
    'MIS':'경영정보학',
    '경영(MIS)':'경영정보학',
    '정보처리(MIS)':'경영정보학',
    '경제학과':'경제학',
    '경제':'경제학',
    '경제학부':'경제학',
    '경제과':'경제학',
    '경제학사':'경제학',
    '컴퓨터공학과':'컴퓨터공학',
    '컴퓨터':'컴퓨터공학',
    '컴퓨터학과':'컴퓨터공학',
    '컴퓨터과학':'컴퓨터공학',
    '컴퓨터학':'컴퓨터공학',
    '컴퓨터과학과':'컴퓨터공학',
    '컴퓨터 공학':'컴퓨터공학',
    '컴퓨터공':'컴퓨터공학',
    '컴퓨터 공학과':'컴퓨터공학',
    '컴퓨터 과학과':'컴퓨터공학',
    '컴퓨터과':'컴퓨터공학',
    '컴퓨터공학전공':'컴퓨터공학',
    '통신컴퓨터공학부 컴퓨터전공':'컴퓨터공학',
    '컴퓨터 학부':'컴퓨터공학',
    '통신`컴퓨터공학분 컴퓨터전공':'컴퓨터공학',
    '전자공학과':'전기전자공학',
    '전기전자공학과':'전기전자공학',
    '전자공학':'전기전자공학',
    '전기전자공학부':'전기전자공학',
    '전자공학부':'전기전자공학',
    '공대 전기전자공학과':'전기전자공학',
    '기계전자공학부 전자 전공':'전기전자공학',
    '영어영문':'영어영문학',
    '영어영문학과':'영어영문학',
    '영어학과':'영어영문학',
    '영어학':'영어영문학',
    '영어과':'영어영문학',
    '영어영문과':'영어영문학',
    '영어 영문학':'영어영문학',
    '영어 영문학과':'영어영문학',
    '영어 영문':'영어영문학',
    '영어영문학부':'영어영문학',
    '영어 영문과':'영어영문학',
    '영어영문학(야)':'영어영문학',
    '영어영문학사':'영어영문학',
    '영문영어':'영어영문학',
    '영어 전공':'영어영문학',
    '기계공학과':'기계공학',
    '기계공학부':'기계공학',
    '기계':'기계공학',
    '기계공':'기계공학',
    '기계과':'기계공학',
    '기계 공학과':'기계공학',
    '기계공악':'기계공학',
    '기계 공학':'기계공학',
    '기계공학 전공':'기계공학',
    '기계고학부':'기계공학',
    '기계공확과':'기계공학',
    '산업공학과':'산업공학',
    '산업공학전공':'산업공학',
    '회계학과':'회계학',
    '회계':'회계학',
    '회계헉':'회계학',
    '회계학과(야)':'회계학',
    '상경학부(회계전공)':'회계학',
    '경영학부(회계학)':'회계학',
    '회계확과':'회계학',
    '회계학과졸업':'회계학',
    '회계학':'회계학',
    '경영경제':'경영경제학',
    '경영경제 (주)':'경영경제학',
    '경영경제 전공':'경영경제학',
    '경영경제학과':'경영경제학',
    '행정학과':'행정학',
    '행정':'행정학',
    '행정학부':'행정학',
    '행정과':'행정학',
    '행정학전공':'행정학',
    '행정학과 졸업':'행정학',
    '법정대 행정학':'행정학',
    '법과대학 행정학':'행정학',
})

df['근무형태']=df['근무형태'].replace(
{
'정규직, 계약직, ':'정규직, 계약직',
'정규직, ':'정규직',
'정규직, 계약직, 해외취업, ':'정규직, 계약직, 해외취업',
'정규직, 계약직, 파견직, ':'정규직, 계약직, 파견직',
'정규직, 계약직, 해외취업, 파견직, ':'정규직, 계약직, 해외취업, 파견직',
'계약직, ':'계약직',
'정규직, 계약직, 해외취업, 인턴, ':'정규직, 계약직, 해외취업, 인턴',
'정규직, 해외취업, ':'정규직, 해외취업',
'정규직, 계약직, 인턴, 파견직, ':'정규직, 계약직, 인턴, 파견직',
'정규직, 계약직, 인턴, ':'정규직, 계약직, 인턴',
'정규직, 파견직, ':'정규직, 파견직',
'인턴, ':'인턴',
'정규직, 계약직, 해외취업, 인턴, 파견직, ':'정규직, 계약직, 해외취업, 인턴, 파견직',
'정규직, 해외취업, 파견직, ':'정규직, 해외취업, 파견직',
'계약직, 파견직, ':'계약직, 파견직',
'계약직, 인턴, ':'계약직, 인턴',
'정규직, 인턴, ':'정규직, 인턴',
'계약직, 해외취업, 파견직, ':'계약직, 해외취업, 파견직',
'정규직, 해외취업, 인턴, ':'정규직, 해외취업, 인턴',
'정규직, 계약직, 해외취업, 병역특례, 인턴, 파견직, ':'정규직, 계약직, 해외취업, 병역특례, 인턴, 파견직',
})

df['연']=df['근무경력'].str.split(" ").str[0]
df['개월']=df['근무경력'].str.split(" ").str[-1]
df1=pd.concat([df.연,df.개월],axis=1)

df1['연'].loc[df1['연'] == '0개월'] = '0년'
df1['연'].loc[df1['연'] == '1개월'] = '0년'
df1['연'].loc[df1['연'] == '2개월'] = '0년'
df1['연'].loc[df1['연'] == '3개월'] = '0년'
df1['연'].loc[df1['연'] == '4개월'] = '0년'
df1['연'].loc[df1['연'] == '5개월'] = '0년'
df1['연'].loc[df1['연'] == '6개월'] = '0년'
df1['연'].loc[df1['연'] == '7개월'] = '0년'
df1['연'].loc[df1['연'] == '8개월'] = '0년'
df1['연'].loc[df1['연'] == '9개월'] = '0년'
df1['연'].loc[df1['연'] == '10개월'] = '0년'
df1['연'].loc[df1['연'] == '11개월'] = '0년'

df1['A'] = df1['연'].str.rstrip("년")
df1['B'] = df1['개월'].str.rstrip("개월")
df1['연차를 개월로 환산'] = df1['A'].astype(dtype='int64')*12
df1['B'] = df1['B'].astype(dtype='int64')*1
df['근무경력'] = df1['연차를 개월로 환산']+df1['B']
df=df.drop(['연'],axis=1)
df=df.drop(['개월'],axis=1)

X_train = df[:16570]
X_test = df[16570:]

In [7]:
numeric_features = ['대학성적','근무경력']
categorical_features = ['직종','세부직종','직무태그','근무형태','근무지역','출신대학','대학전공','어학시험','자격증']

X_train = X_train[numeric_features+categorical_features]
X_test = X_test[numeric_features+categorical_features]

In [8]:
X_train['직무태그']=X_train['직무태그'].fillna(X_train['세부직종'][0])
X_test['직무태그']=X_test['직무태그'].fillna(X_test['세부직종'][0])

In [9]:
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=True)),
        #("encoder", TargetEncoder(smoothing=10, min_samples_leaf=15)),
        #("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=100)),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", Ridge(alpha=15)),  #작으면 작을수록 과적합이 커진다. 적정값은 튜닝을 통해 알아내야한다.
    ]
)

In [10]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5) #사이킷런 rmse 제공 x -> mse를 돌리고 neg_으로 -1을 곱해준다 

print("Default LM CV scores: ", np.sqrt(-1*scores))
print("Default LM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

Default LM CV scores:  [810.20492865 798.03003084 896.73549577 957.01063898 989.44939576]
Default LM CV mean = 893.57 with std = 369.01


In [11]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean","median","most_frequent"]),
    "preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.01},{'q':0.02},{'q':0.03},{'q':0.04},{'q':0.05},{'q':0.06},{'q':0.07},{'q':0.08},{'q':0.09},{'q':0.1}]),
    #"preprocessor__column__cat__encoder__smoothing": IntDistribution(1,100),
    #"preprocessor__column__cat__encoder__min_samples_leaf": IntDistribution(1,100),
    #"preprocessor__column__cat__poly__degree": IntDistribution(1,5),
    "preprocessor__selector__percentile": IntDistribution(50,100), 
    #"classifier__alpha": IntDistribution(1,20)
}

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search = OptunaSearchCV(model, param_distributions, cv=5, 
                               scoring='neg_mean_squared_error', n_trials=60,
                               study=optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction='maximize'))
optuna_search.fit(X_train, y_train)

print(f"\nBest params: {optuna_search.best_params_}")
print(f"\nBest score: {np.sqrt(-1*optuna_search.best_score_):.2f}")


Best params: {'preprocessor__column__num__imputer__strategy': 'median', 'preprocessor__column__num__outlier__kw_args': {'q': 0.02}, 'preprocessor__selector__percentile': 98}

Best score: 890.12
CPU times: total: 3min 9s
Wall time: 31.6 s


In [12]:
# 옵튜나는 다시 모델링을 해야한다.
# 최적화된 하이퍼파라미터로 파이프라인 재설정

model.set_params(**optuna_search.best_params_)

# OOF Prediction
models = cross_validate(model, 
                        X_train, y_train, 
                        cv=5, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

#5번의 cv를 통해서 얻은 값들을 평균냄

scores = models['test_score']
print("\nTuned LM CV scores: ", np.sqrt(-1*scores))
print("Tuned LM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))


Tuned LM CV scores:  [802.31835251 787.02287093 898.36977395 947.97847491 996.33854404]
Tuned LM CV mean = 890.12 with std = 379.45


In [13]:
###### submission 화일 생성
filename = f'LM_{LM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)

<font color="#CC3D3D"><p>
# End